<a href="https://colab.research.google.com/github/Johyeonje/DeepLearningStudy/blob/master/Start!_chap5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Image

#분류

##이항분류

###화이트 와인과 레드 와인의 분류

와인 데이터 셋 불러오기

In [ ]:
import pandas as pd
red = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
print(red.head())
print(white.head())

12개의 속성을 가지고 화이트와인인지 레드와인인지 구별하는 작업이다.    
먼저 레드와인과 화이트와인인지 구별하는 속성을 추가해준다.

In [ ]:
red['type'] = 0
white['type'] = 1
print(red.head(2))
print(white.head(2))

wine = pd.concat([red, white])
print(wine.describe())

#####describe()
데이터의 간단한 통계정보를 확인하는 함수 describe()를 사용하면 평균값인 mean이    
 0.75인걸 확인할 수 있는데 화이트와인을 뜻하는 1이 더 많다는 걸 알 수 있다.    
 아래는 간단한 그래프를 그려본 것이다.

In [ ]:
import matplotlib.pyplot as plt
plt.hist(wine['type'])
plt.xticks([0,1])
plt.show()

print(wine['type'].value_counts())

#####xticks
: 눈금

#####info()
외부 데이터의 경우 구성을 알기 어려운데 이 때 사용하는 함수가 info()

In [ ]:
print(wine.info())

####수치들의 정규화

각 속성들이 저장된 자료의 단위가 다 다르기 때문에 속성별로 고르게 가중치에 영향을 주게 하기 위해서 정규화를 한다.

In [ ]:
wine_norm = (wine - wine.min()) / (wine.max() - wine.min())
print(wine_norm.head())
print(wine_norm.describe())

####셔플 후 np.array

In [ ]:
import numpy as np
wine_shuffle = wine_norm.sample(frac=1)
print(wine_shuffle.head())
wine_np = wine_shuffle.to_numpy()
print(wine_np[:5])

####훈련과 검증 데이터의 분리

to_categorical : 분류 문제에 자주 쓰이는 함수로 정답 행렬을 원-핫 인코딩 방식으로 바꿈    
원-핫 인코딩(One-Hot Encoding) : 정답 인덱스에만 1 나머지는 다 0    
num_classes : 정답 클래스의 갯수


In [ ]:
import tensorflow as tf
train_idx = int(len(wine_np)*0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]
print(train_X[0])
print(train_Y[0])
print(test_X[0])
print(test_Y[0])
train_Y = tf.keras.utils.to_categorical(train_Y, 
                                        num_classes=2)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=2)
print(train_Y[0])
print(test_Y[0])

####모델 구성

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
            tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
            tf.keras.layers.Dense(units=24, activation='relu'),
            tf.keras.layers.Dense(units=2, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

####softmax 활성함수



분류에서 가장 만이 쓰이는 함수로 출력 값들을 자연 로그의 및인 $e$의 지수로 사용해 계산한 뒤 모두 더한 값으로 나눈다.    
이렇게 나온 결괏값들은 총합이 1인 확률값이 된다.    
$P(z) = \frac{e^{z_j}}{\sum^K_{k=1}e^{zk}}(for　i =  1,2,...,K)$

예를 들어, [2, 1, 0]이라는 값이 있을 경우 소프트맥스로 변환한다면    
$sum = \sum^K_{k=1}e^{z_k}=e^2 + e^1 + e^0=11.1073$    
        
$sum = \frac{e^{z_j}}{\sum^K_{k=1}e^{z_k}} = [\frac{e^2}{sum},\frac{e^1}{sum},\frac{e^0}{sum}] = [0.67, 0.24, 0.09]$    
다음과 같은 확률값이 된다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/pic5_5.png', width=500)

소프트맥스는 분류 문제나 언어 RNN에서의 다음 토큰 예측, 강화학습에서 에이전트의 행동 확률을 구하는 등 결괏값으로 확률이 필요한 다양한 분야에서 쓰인다. 여기서는 예측값이 레드 와인일 확률과 화이트 와인일 확률을 구하는데 쓰인다. 위 분류 네트워크의 결괏값은 [0.97, 0.03]처럼 합이 1.0인 확률값이 된다.

####엔트로피(entropy)와 정보이론
정보이론에서는 정보량을 나타내기 위해 엔트로피라는 단위를 사용한다. 엔트로피는 확률의 역수에 로그를 취한 값이다. 어떤 사건 X의 확률이 $p(x)$, 엔트로피를 $h(x)$라고 할 때, 엔트로피의 식은 아래와 같다.    
    
$h(x)=log\frac{1}{p(x)}=-logp(x)$    
    
확률을 역수로 취해주는 것은 확률이 높은 사건일수록 정보량(놀라움)이 적다고 판단하기 때문이다. 내일 비가 올 확률이 1%일 때 비가 오지 않을 확률은 99%일 것이다. 이때 각 사건의 정보량은 다음과 같다.    
    
$h(비)=-log0.01=4.605$    
$h(비가오지않음)=-log0.99=0.010$    
    
비가 오지 않는 경우는 비가 오는 경우보다 460배 정도 더 놀라운 사건이 된다.    
엔트로피의 기댓값은 각 엔트로피에 확률을 곱해준 값이다. 통계에서 기댓값의 정의는 각 사건이 벌어졌을 때의 이득과 그 사건이 벌어질 확률을 곱한 값이기 때문이다. 이를 수식으로 나타내면 아래와 같다.    
    
$E(X)=-p(x)logp(x)$    
    
위의 비 사건의 엔트로피를 계산하면 0.0461, 비가 안오는 사건의 엔트로피는 0.0099로 비가 오는 사건의 엔트로피가 높은 것을 확인할 수 있다. 이 경우 엔트로피의 양만큼 비가 올 확률에 대한 예측값을 조정해서, 결과적으로 다음에 얻을 엔트로피를 낮추려고 노력한다.    
`높은 엔트로피는 높은 불확실성을 의미하기 때문에 엔트로피를 줄이면 불확실성이 낮아지고 의미있는 정보를 얻을 수 있다.`


손실 함수인 loss에 자주 쓰이던 mse 대신 categorical_crossentropy라는 낮선 값이 들어왔다. categorical은 범주에 쓰이는 분류라는 뜻이다. 크로스 엔트로피(crossentropy)는 엔트로피의 기댓값과 비슷하지만 약간 다른 점이 있다.

확률 부분에 $p(x)$를 곱하는 것과 달리 로그 뒤에는 $q(x)$를 곱한다. 여기서 $q(x)$란 분류 네트워크가 예측한 라벨의 확률 값이다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/crossentropy.png', width=600)

그림 5.6에서 볼 수 있듯이 분류 문제에서 $p(x)$와 $q(x)$에 해당하는 범주의 종류는 2개 이상이다. 이에 대한 해결책은 `모든 범주에 대한 평균을 내는 것`이다. 범주의 수를 n이라고 할 때 범주에 대한 크로스엔트로피(Categorical Cross Entropy, CCE)의 식은 아래와 같다.

$CCE=-\frac{1}{n}\sum^n_{j=1}p(x)logq(x)$

이 식에 따라 계산해보면 그림 5.5의 CCE는 다음과 같다.

$CCE_1=-\frac{1}{2}(1.0*log0.87+0.0*log0.131)=0.696$

이 값만 보면 CCE가 높은것인지 낮은 것인지 가늠하기 힘들다. 비교를 하기 위해 다른 경우를 가정해 보겠다.    
정답이 화이트 와인일 때 레드 와인일 확률을 0.6, 화이트 와인일 확률을 0.4로 예측하는 경우가 있다면 정답보다 오답에 가깝게 예측하기 때문에 그림 5.5보다 예측을 못하는 경우라고 생각해볼 수 있다. 실제로 그러한지 계산해보자.

$CCE_2=-\frac{1}{2}(0.0*log0.6+1.0*log0.4)=0.4581$

두 번째 경우의 CCE 수치는 0.4581로, 그림 5.5의 0.0696보다 6.5배 정도 높아졌다. CCE도 손실의 일종이기 때문에 낮을수록 좋은 값이고 네트워크는 손실을 낮추기 위해서 노력한다. 따라서 CCE가 낮을 수록 예측을 잘하는 네트워크가 된다.

또 한 가지 알아야 할 점은 정답 라벨은 늘 1.0이고 오답 라벨은 언제나 0.0이기 때문에 오답 라벨에 곱해지는 값은 버려진다는 점이다. 위 식들은 아래처럼 정답 라벨에 관한 예측만 남겨도 값이 달라지지 않는다.

$CCE_1=-\frac{1}{2}(1.0*log0.87)=0.696$

$CCE_2=-\frac{1}{2}(1.0*log0.4)=0.4581$

그리고 model.copile 함수의 metrics 인수에 정확도(accuracy)가 추가됐다. 분류 문제는 정확도로 퍼포먼스를 측정하기 때문에 정확도를 설정하는 것은 필수이다.

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, batch_size=32, validation_split=0.25)

#####batch_size, epoch
batch_size는 보편적으로 32로 한다.
epoch는 데이터셋 전체를 한번 학습했을때 1이다.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()


In [ ]:
model.evaluate(test_X, test_Y)

##다항 분류

 : 범주가 2개를 초과하는 경우이다.    
 와인 데이터 세트에 와인의 품질은 0에서 10까지 숫자이기 때문에 다항 분류의 범수에 속한다. 와인의 품질을 예측해보자.    
 일단 품질 데이터가 어떤 비율로 구성되는지 알아보자.


In [ ]:
print(wine['quality'].describe())
print(wine['quality'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
plt.hist(wine['quality'], bins=7, rwidth=0.8)
plt.show()

그래프 처럼 데이터간의 분포 차이가 크기 때문에 3~5를 나쁨, 6을 보통, 7~9를 좋음으로 재분류 하도록 한다.

In [ ]:
wine.loc[wine['quality']<=5, 'new_quality'] = 0
wine.loc[wine['quality']==6, 'new_quality'] = 1
wine.loc[wine['quality']>=7, 'new_quality'] = 2
print(wine['new_quality'].describe())
print(wine['new_quality'].value_counts())

#####loc 
: 데이터프레임에 특정한 인덱스를 골라내는 역할.    
대괄호 안에 인수를 하나만 넣으면 행을 골라내고, 쉼표를 포함한 두개의 인수를 넣으면 차례대로 행, 열을 골라낸다.

In [ ]:
data = [['Apple',11],['Banana;,23'],['Coconut',35]]
df = pd.DataFrame(data, columns=['Fruit', 'Count'])
print(df)
print()
print(df.loc[0])    #첫 번째 행 선택
print()
print(df.loc[0,'Fruit'])    #첫 번째 행의 Fruit 열을 선택

데이터 정규화 및 훈련 데이터와 테스트 데이터의 분리

In [ ]:
del wine['quality']
wine_norm = (wine-wine.min()) / (wine.max() - wine.min())
wine_shuffle = wine_norm.sample(frac=1)
wine_np = wine_shuffle.to_numpy()

train_idx = int(len(wine_np) * 0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=3)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=3)

예측하고자 하는 new_quality가 quality로 부터 만들어진 것이기 때문에 quality를 테이블에서 삭제한 뒤 남은 데이터를 가지고 예측을 진행한다.

In [ ]:
model = tf.keras.Sequential([
              tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
              tf.keras.layers.Dense(units=24, activation='relu'),
              tf.keras.layers.Dense(units=12, activation='relu'),
              tf.keras.layers.Dense(units=3, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.03), loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_Y, epochs=25, batch_size=32, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()


In [ ]:
model.evaluate(test_X, test_Y)

##Fashion MNIST

MNIST : 손으로 쓴 숫자 글씨를 모아놓은 데이터 세트이다.    
Fashion MNIST : 손글씨와 유사하지만 의류에 대한 데이터 세트이다.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

print(len(train_X), len(test_X))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_X[0], cmap='gray')
plt.colorbar()
plt.show()

print(train_Y[0])

In [ ]:
train_X = train_X / 255.0
test_X = test_X / 255.0

print(train_X[0])

In [ ]:
model = tf.keras.Sequential([
                    tf.keras.layers.Flatten(input_shape=(28,28)),
                    tf.keras.layers.Dense(units=128, activation='relu'),
                    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

sparse_categorical_crossentropy : 출력층이 여러층이 되면 모든 층을 표현하는게 원핫인코딩을 쓰기에 너무 메모리가 낭비되기 때문에 index로 표시하겠다.    
ex) [0,0,0,0,1] = 4 로 표기

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()


In [ ]:
model.evaluate(test_X, test_Y)